## ReTap - UPDRS-Tapping Assessment - Predictions

This notebooks investigates optimal hand- and fingertapping algorithms as part of the 
ReTune-Dyskinesia project.



### 0. Loading packages and functions, defining paths



In [2]:
# Importing Python and external packages
import os
import sys
import importlib
import json
import pandas as pd
import numpy as np
import sklearn as sk
import scipy
import matplotlib.pyplot as plt
# import matplotlib.gridspec as gridspec

import seaborn as sns
from scipy import signal
from scipy import stats
from array import array
import datetime as dt
from dataclasses import  dataclass, field
from itertools import compress
from typing import Any

In [3]:
# check some package versions for documentation and reproducability
print('Python sys', sys.version)
print('pandas', pd.__version__)
print('numpy', np.__version__)
# print('mne_bids', mne_bids.__version__)
# print('mne', mne.__version__)
print('sci-py', scipy.__version__)
print('sci-kit learn', sk.__version__)


## developed with:
# Python sys 3.9.7 (default, Sep 16 2021, 08:50:36) 
# [Clang 10.0.0 ]
# pandas 1.3.4
# numpy 1.20.3
# mne_bids 0.9
# mne 0.24.1
# sci-py 1.7.1
# sci-kit learn 1.0.1

## Currently (own env) since 31.08.22
# Python sys 3.9.12 (main, Jun  1 2022, 06:36:29) 
# [Clang 12.0.0 ]
# pandas 1.4.3
# numpy 1.21.5
# sci-py 1.7.3
# sci-kit learn 1.1.1

Python sys 3.9.13 (main, Oct 13 2022, 21:23:06) [MSC v.1916 64 bit (AMD64)]
pandas 1.4.4
numpy 1.23.3
sci-py 1.9.1
sci-kit learn 1.1.2


In [4]:
# own functions
from retap_utils import utils_dataManagement
import retap_utils.get_datasplit as get_split

import tap_predict.tap_pred_prepare as pred_prep
import tap_plotting.retap_plot_clusters as plot_cluster

## 1) Split development and hold-out-test data sets

- Development data is used to train and test the model using iterative cross-validation
- Hold-out test data is NOT USED at all during cross-validation, and will be used to test the trained model as an external validation

### 1a. Import extracted Features

In [5]:
### IMPORT CREATED CLASSES FROM FILES
from tap_extract_fts.main_featExtractionClass import FeatureSet, singleTrace

# define path with feature class
deriv_path = os.path.join(utils_dataManagement.get_local_proj_dir(), 'data', 'derivatives')

ftClass = utils_dataManagement.load_class_pickle(os.path.join(deriv_path, 'ftClass_ALL_20221214.P'))
ftClass10 = utils_dataManagement.load_class_pickle(os.path.join(deriv_path, 'ftClass_ALL_max10_20221214.P'))

## 2) ML-dataset Preparation

#### 2a. Including ALL features

In [16]:
importlib.reload(pred_prep)

traces, feats = pred_prep.select_traces_and_feats(
    ftClass,
    center='all',
    use_sel_fts=True,
)
X, y = pred_prep.create_X_y_vectors(
    ftClass,
    incl_traces=traces,
    incl_feats=feats,
    to_norm=False,
)

### 2b. Ensemble method, start with clustering on intraTapInterval and overall tapping-frequency

Create X1 with selected input features (mean and coef of variation of intra-tap-interval) and
overall tapping frequency to find two clusters (y_clusters) dividing fast vs slow tappers. 

In [61]:
importlib.reload(pred_prep)
importlib.reload(get_split)
importlib.reload(plot_cluster)

# set variables for pre-clustering
ftClass_to_use = ftClass10
n_clusters = 2
traces_excl = [
    'DUS006_M0S0_L_1',
]
ft_sel = [
    'mean_intraTapInt',
    'coefVar_intraTapInt',
    'freq'
]
to_mask_4 = True
to_mask_0 = False
to_zscore = True
to_norm = False

# get dict with dev and hold-out datasets
datasplit_subs = get_split.find_dev_holdout_split(
    feats=ftClass_to_use, )

# create dataclass for clustering (input matrix, label vector)
# only include dev, exclude hold-out
dev_data = pred_prep.create_X_y_vectors(
    ftClass=ftClass_to_use,
    incl_feats=ft_sel,
    incl_traces=ftClass_to_use.incl_traces,
    excl_traces=traces_excl,
    excl_subs=datasplit_subs['hout'],  # excl hold out data
    to_zscore=to_zscore,
    to_norm=to_norm,
    to_mask_4=to_mask_4,
    to_mask_0=to_mask_0,
    return_ids=True,
    as_class=True
)

# create cluster labels
y_clust, centr_clust, _ = plot_cluster.get_kMeans_clusters(
    X=dev_data.X,
    n_clusters=n_clusters,
    use_pca=True,
    to_zscore=to_zscore,
    to_norm=to_norm,
)

# Define which cluster contains faster tappers
cluster_mean_ITIs = []

ft = 'mean_intraTapInt'
if to_zscore: print(f'Mean {ft} (z-scored):')
elif to_norm: print(f'Mean {ft} (normed):')

for i_cls in np.unique(y_clust):

    i_ft = np.where([f == ft for f in ft_sel])[0][0]
    mean_iti_cluster = np.mean(dev_data.X[y_clust == i_cls, i_ft])
    cluster_mean_ITIs.append(mean_iti_cluster)

    print(f'\tcluster {i_cls}: {mean_iti_cluster}')

fast_cluster_i = np.argmin(cluster_mean_ITIs)
if fast_cluster_i == 0: slow_cluster_i = 1
if fast_cluster_i == 1: slow_cluster_i = 0

print(f'Fast tappers are clustered in cluster index {fast_cluster_i}')
print(f'Slow tappers are clustered in cluster index {slow_cluster_i}')

SPLITTING DATA IN DEV AND HOLD-OUT
Original score distribution: {0: 40, 1: 154, 2: 122, 3: 57, 4: 3}
Original score %: {0: 10.6, 1: 41.0, 2: 32.4, 3: 15.2, 4: 0.8}
Accepted Split: random state 63

Resulting distributions in splitted data sets:

	dev data set (n = 285):
score 0: # 32 (11 %)
score 1: # 115 (40 %)
score 2: # 94 (33 %)
score 3: # 42 (15 %)
score 4: # 2 (1 %)
	hout data set (n = 91):
score 0: # 8 (9 %)
score 1: # 39 (43 %)
score 2: # 28 (31 %)
score 3: # 15 (16 %)
score 4: # 1 (1 %)
# of NaNs per feat: [0 0 0]


c:\Users\habetsj\Anaconda3\envs\retap\lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


Mean mean_intraTapInt (z-scored):
	cluster 0: -0.357289281961114
	cluster 1: 1.4546777908416781
Fast tappers are clustered in cluster index 0
Slow tappers are clustered in cluster index 1


### Make X_2 input Matrix with more features for score-prediction per Cluster

In [62]:
importlib.reload(pred_prep)

incl_traces, ft_list = pred_prep.select_traces_and_feats(
    ftClass_to_use, use_sel_fts=True, excl_traces=traces_excl,
)

feats_for_2nd_pred = [
    # 'freq',
    'coefVar_intraTapInt',
    # 'mean_intraTapInt',
    # 'slope_intraTapInt',
    'decr_intraTapInt',
    'mean_tapRMS',
    'coefVar_tapRMS',
    # 'slope_tapRMS',
    'decr_tapRMS',
    'mean_raise_velocity',
    'jerkiness_trace'
]

cv_data = pred_prep.create_X_y_vectors(
    ftClass_to_use,
    incl_traces=ftClass_to_use.incl_traces,
    incl_feats=feats_for_2nd_pred,
    excl_traces=traces_excl,
    excl_subs=datasplit_subs['hout'],  # due to hold out data set
    to_norm=to_norm,
    to_zscore=to_zscore,
    to_mask_4=to_mask_4,
    return_ids=True,
    as_class=True,
)

# create final dataframe with true and ensemble-predicted labels
# default all NaN's, filled during ensemble prediction
overall_perf = pd.DataFrame(
    data=np.array([[np.nan] * len(cv_data.y)] * 2).T,
    columns=['y_true', 'y_pred'],
    index=cv_data.ids,
)
overall_perf['y_true'] = cv_data.y

# of NaNs per feat: [0 0 0 0 3 0 0]


### Split input matrix X_2 in two generated clusters:
- split X and y in two groups based on clusters
- test default ML modeling on both groups

In [63]:
print(f'Total CV X shape: {cv_data.X.shape}')

cv_fast_data = pred_prep.predictionData(
    X=cv_data.X[y_clust == fast_cluster_i],
    y=cv_data.y[y_clust == fast_cluster_i],
    ids=cv_data.ids[y_clust == fast_cluster_i])

cv_slow_data = pred_prep.predictionData(
    X=cv_data.X[y_clust == slow_cluster_i],
    y=cv_data.y[y_clust == slow_cluster_i],
    ids=cv_data.ids[y_clust == slow_cluster_i])

print(f'Fast X shape: {cv_fast_data.X.shape}, Slow X shape: {cv_slow_data.X.shape}')



Total CV X shape: (284, 7)
Fast X shape: (228, 7), Slow X shape: (56, 7)


### Visualise features in specific clusters

In [65]:
# create lists for boxplots of features per subscore, per cluster

temp_data = cv_fast_data

box_lists = {}
for f in range(temp_data.X.shape[1]):
    box_lists[f] = {}
    for i in range(4): box_lists[f][i] = []


for i in np.arange(temp_data.X.shape[0]):

    score = temp_data.y[i]

    for f in range(temp_data.X.shape[1]):

        box_lists[f][int(score)].append(temp_data.X[i, f])

# plot features within cluster, and decide on strategy
# pm: use pre-knowledge about clusters
# likelihood in faster cluster for 1-2 scores
# use probabilities and adapt the threshold for acceptance
# start finding border scores (e.g. 1 or 3)

for i_f, ft in enumerate(feats_for_2nd_pred):

    plot_lists = [box_lists[i_f][i] for i in range(4)]

    plt.boxplot(plot_lists)
    plt.title(ft)
    plt.xticks(range(1, len(plot_lists) + 1), labels=['0', '1', '2', '3+4'])
    plt.xlabel('UPDRS tap-score')
    plt.ylabel('Z-score (a.u.)')
    plt.close()

### Test different prediction models for second step in Fast Cluster

optimal thresholds (to prevent too large False Positive Values)
#### predicting the best tappers (0-1)
- (best) RandomForest, cutoff .75 (TPR ~ .75-.8, FPR ~ .15)
- .58 - .6 for LogReg
- .6 for svm linear kernel
- .6 for svm poly kernel

#### indicating updrs 3 chance for next step
- .15 for log reg and lda (svc not succesful)

In [179]:
from retap_utils.plot_helpers import remove_duplicate_legend
from tap_predict import retap_cv_models as cv_models
from tap_plotting import plot_cv_folds as plot_folds

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold


from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

from sklearn.metrics import (
    confusion_matrix, roc_auc_score, roc_curve,
    accuracy_score, f1_score, precision_score,
    recall_score, plot_roc_curve, plot_confusion_matrix
)


from sklearn.metrics import classification_report
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import auc
from sklearn.metrics import RocCurveDisplay, ConfusionMatrixDisplay


In [ ]:
# tn, fp, fn, tp = confusion_matrix(y_true,y_preds).ravel()
# # add outcomes to dedicated lists
# ['Accuracy'].append(accuracy_score(y_true,y_preds))
# ['AUROC'].append(roc_auc_score(y_true, y_probas[1]))
# ['F1_score'].append(f1_score(y_true,y_preds))
# ['Precision'].append(precision_score(y_true, y_preds)) # precision/PPV
# ['Recall'].append(recall_score(y_true, y_preds)) # sensitivity/recall/TPR
# ['FPR'].append(fp / (fp+tn)) # false-positive, false-alarm rate

In [324]:
importlib.reload(cv_models)
importlib.reload(plot_folds)

# CLassification Settings
temp_data = cv_fast_data  # data to use here
score_to_predict = 1
clf_choice = 'RF'
nFolds = 4
to_plot = True

multiclass = True

if multiclass:
    y_pred_true = temp_data.y
    mc_labels = ['0', '1', '2', '3-4']

else:
    if score_to_predict == 1:
        y_pred_true = temp_data.y <= score_to_predict
        plot_thresholds = [.65, .7, .75]
        roc_title = f'Identify UPDRS 0/1 vs Rest ({clf_choice})'

    elif score_to_predict == 3:
        y_pred_true = temp_data.y == score_to_predict
        plot_thresholds = [.25, .4, .5]
        roc_title = f'Identify UPDRS 3-4 vs Rest ({clf_choice})'


chance = round(sum(y_pred_true) / len(y_pred_true), 3)
print(f'shape of X_dev: {temp_data.X.shape}')
print(f'predicting UPDRS score {score_to_predict}')
print(f'total y true: {sum(y_pred_true)}')
print(f'Chance level: {chance}')

(y_pred_dict, y_proba_dict,
 y_true_dict, og_pred_idx
) = cv_models.get_cvFold_predictions_dicts(
    X_cv=temp_data.X,
    y_cv=y_pred_true,
    cv_method=StratifiedKFold,
    n_folds=nFolds,
    clf=clf_choice,
)
if to_plot and not multiclass: 
    plot_folds.plot_ROC_AUC_confMatrices_for_folds(
        y_true_dict=y_true_dict,
        y_proba_dict=y_proba_dict,
        plot_thresholds=plot_thresholds,
        roc_title=roc_title,
    )
if multiclass:
    cm = cv_models.multiclass_conf_matrix(
        y_true=y_true_dict, y_pred=y_pred_dict,
        labels=mc_labels,
    )
    # print(f'\nConfusion Matrix:\n{cm}')
    # mean_pen, std_pen, _ = cv_models.get_penalties_from_conf_matr(cm)
    # print(f'mean UPDRS-penalty: {round(mean_pen, 2)}'
    #         f' (+/- {round(std_pen, 2)})')



shape of X_dev: (228, 7)
predicting UPDRS score 1
total y true: 324
Chance level: 1.421
RandomForestClassifier(class_weight='balanced', min_samples_split=5,
                       n_estimators=500, random_state=27)
Fold 0: # of samples: train 171, test 57
Fold 1: # of samples: train 171, test 57
Fold 2: # of samples: train 171, test 57
Fold 3: # of samples: train 171, test 57


### Extract best tappers (0-1 predicted) from fast-tappers and classify remaining part

In [109]:
importlib.reload(pred_prep)

# Set Final Prediction best fast-tappers to 1
set_outcome = True
data_fast_01, data_fast_rest = pred_prep.split_dataset_on_pred_proba(
    orig_dataset=cv_fast_data,
    probas=y_proba_dict,
    og_indices=og_pred_idx,
    proba_thr=.75,
)
if set_outcome:
    for trace_id in data_fast_01.ids:
        overall_perf.at[trace_id, 'y_pred'] = 1


#### Identify UPDRS 3 in remaining fast tappers

- use positive prediction for UPDRS III in previous step (which is not used for splitting data)
- opt logreg threshold for updrs-3: .18
- opt lda threshold for updrs-3: .3

In [325]:
### INCLUDE UPDRS 3 LABELS FROM RPEVIOUS TEP

importlib.reload(cv_models)
n_samples = len(data_fast_rest.ids)
print(f'Remaining # of traces: {n_samples}')
y_scores, counts = np.unique(data_fast_rest.y, return_counts=True)
for s, c in zip(y_scores, counts):
    print(f'\tscore {s}: # {c} ({round(c / n_samples * 100)} %)')


multiclass = True
clf_choice = 'rf'
to_plot=True
mask_0 = True

y_model = data_fast_rest.y.copy()
if mask_0: # mask 0's to 1
    y_model[y_model == 0] = 1
    mc_labels = ['0-1', '2', '3-4']
else:
    mc_labels = ['0', '1', '2', '3-4']

if not multiclass:
    y_model = y_model >= 2

plot_thresholds=[.5, .6, .7]
roc_title=f'Rest Fast-Tappers == UPDRS 2/3 ({clf_choice})'


(cv_pred, cv_proba, cv_true, cv_idx
) = cv_models.get_cvFold_predictions_dicts(
    X_cv=data_fast_rest.X,
    y_cv=y_model,
    n_folds=3,
    clf=clf_choice,
)

if clf_choice == 'lda': thresh3 = .3
elif clf_choice == 'logreg': thresh3 = .3

ids_pos_3 = []
for fold in cv_proba:
    for i_prob, proba in enumerate(cv_proba[fold]):
        if proba[1] > thresh3:
            og_i = cv_idx[fold][i_prob]
            ids_pos_3.append(
                data_fast_rest.ids[og_i]
            )

if to_plot and not multiclass: 
    plot_folds.plot_ROC_AUC_confMatrices_for_folds(
        y_true_dict=cv_true,
        y_proba_dict=cv_proba,
        plot_thresholds=plot_thresholds,
        roc_title=roc_title,
        incl_mean_ROC=True,
    )
if multiclass:
    cm = cv_models.multiclass_conf_matrix(
        y_true=cv_true, y_pred=cv_pred,
        labels=mc_labels,
    )
    # print(f'\nConfusion Matrix:\n{cm}')
    # mean_pen, std_pen, _ = cv_models.get_penalties_from_conf_matr(cm)
    # print(f'mean UPDRS-penalty: {round(mean_pen, 2)}'
    #         f' (+/- {round(std_pen, 2)})')

Remaining # of traces: 168
	score 0: # 18 (11 %)
	score 1: # 59 (35 %)
	score 2: # 69 (41 %)
	score 3: # 22 (13 %)
RandomForestClassifier(class_weight='balanced', min_samples_split=5,
                       n_estimators=500, random_state=27)
Fold 0: # of samples: train 112, test 56
Fold 1: # of samples: train 112, test 56
Fold 2: # of samples: train 112, test 56


### Test Slow Tapper Cluster

In [326]:
importlib.reload(cv_models)
importlib.reload(plot_folds)

# CLassification Settings
temp_data = cv_slow_data  # data to use here
clf_choice = 'RF'
nFolds = 3
mask_0 = True
multiclass = True
score_to_predict = 3

y_model = temp_data.y.copy()
if mask_0: # mask 0's to 1
    y_model[y_model == 0] = 1
    mc_labels = ['0-1', '2', '3-4']
else:
    mc_labels = ['0', '1', '2', '3-4']

if not multiclass:
    y_model = y_model == score_to_predict
    to_plot = True

    if score_to_predict == 1:
        plot_thresholds = [.65, .7, .75]
        roc_title = f'Identify UPDRS 0/1 vs Rest ({clf_choice})'

    elif score_to_predict == 3:
        plot_thresholds = [.25, .4, .5]
        roc_title = f'Identify UPDRS 3-4 vs Rest ({clf_choice})'

# print descriptives
n_samples = len(temp_data.ids)
print(f'Included # of traces: {n_samples}')
y_scores, counts = np.unique(y_model, return_counts=True)
for s, c in zip(y_scores, counts):
    print(f'\tscore {s}: # {c} ({round(c / n_samples * 100)} %)')


(y_pred_dict, y_proba_dict,
 y_true_dict, og_pred_idx
) = cv_models.get_cvFold_predictions_dicts(
    X_cv=temp_data.X,
    y_cv=y_model,
    cv_method=StratifiedKFold,
    n_folds=nFolds,
    clf=clf_choice,
)
if to_plot and not multiclass: 
    plot_folds.plot_ROC_AUC_confMatrices_for_folds(
        y_true_dict=y_true_dict,
        y_proba_dict=y_proba_dict,
        plot_thresholds=plot_thresholds,
        roc_title=roc_title,
    )
if multiclass:
    cm = cv_models.multiclass_conf_matrix(
        y_true=y_true_dict, y_pred=y_pred_dict,
        labels=mc_labels,
    )
    # print(f'\nConfusion Matrix:\n{cm}')
    # mean_pen, std_pen, _ = cv_models.get_penalties_from_conf_matr(cm)
    # print(f'mean UPDRS-penalty: {round(mean_pen, 2)}'
    #         f' (+/- {round(std_pen, 2)})')



Included # of traces: 56
	score 1: # 20 (36 %)
	score 2: # 16 (29 %)
	score 3: # 20 (36 %)
RandomForestClassifier(class_weight='balanced', min_samples_split=5,
                       n_estimators=500, random_state=27)
Fold 0: # of samples: train 37, test 19
Fold 1: # of samples: train 37, test 19
Fold 2: # of samples: train 38, test 18


### Test all Tappers (without clustering)

In [327]:
importlib.reload(cv_models)
importlib.reload(plot_folds)

n_permutations = 200

# random_penalties = {'mean': [], 'lists': []}
r_states = np.linspace(0, 1000, n_permutations).astype(int)
# for random_shuffle in r_states: 

# CLassification Settings
temp_data = cv_data  # data to use here
clf_choice = 'RF'
nFolds = 6
mask_0 = False
multiclass = True
score_to_predict = 1

y_model = temp_data.y.copy()
if mask_0: # mask 0's to 1
    y_model[y_model == 0] = 1
    mc_labels = ['0-1', '2', '3-4']
else:
    mc_labels = ['0', '1', '2', '3-4']

if not multiclass:
    y_model = y_model == score_to_predict
    to_plot = True

    if score_to_predict == 1:
        plot_thresholds = [.65, .7, .75]
        roc_title = f'Identify UPDRS 0/1 vs Rest ({clf_choice})'

    elif score_to_predict == 3:
        plot_thresholds = [.25, .4, .5]
        roc_title = f'Identify UPDRS 3-4 vs Rest ({clf_choice})'

# # shuffle true y-labels
# np.random.seed(random_shuffle)
# np.random.shuffle(y_model)

# print descriptives
# n_samples = len(temp_data.ids)
# print(f'Included # of traces: {n_samples}')
# y_scores, counts = np.unique(y_model, return_counts=True)
# for s, c in zip(y_scores, counts):
#     print(f'\tscore {s}: # {c} ({round(c / n_samples * 100)} %)')


(y_pred_dict, y_proba_dict,
y_true_dict, og_pred_idx
) = cv_models.get_cvFold_predictions_dicts(
    X_cv=temp_data.X,
    y_cv=y_model,
    cv_method=StratifiedKFold,
    n_folds=nFolds,
    clf=clf_choice,
    verbose=False,
)
if to_plot and not multiclass: 
    plot_folds.plot_ROC_AUC_confMatrices_for_folds(
        y_true_dict=y_true_dict,
        y_proba_dict=y_proba_dict,
        plot_thresholds=plot_thresholds,
        roc_title=roc_title,
        verbose=False,
    )
if multiclass:
    cm = cv_models.multiclass_conf_matrix(
        y_true=y_true_dict, y_pred=y_pred_dict,
        labels=mc_labels,
    )
    # print(f'\nConfusion Matrix:\n{cm}')
    mean_pen, std_pen, pen_list = cv_models.get_penalties_from_conf_matr(cm)
    # print(f'mean UPDRS-penalty: {round(mean_pen, 2)}'
    #         f' (+/- {round(std_pen, 2)})')
    
    # random_penalties['mean'].append(mean_pen)
    # random_penalties['lists'].append(pen_list)



In [304]:
penalties_full_chance = []
true_labels = overall_perf['y_true'].values

r_states = np.linspace(0, 1000, n_permutations).astype(int)

for r_seed in r_states:
    np.random.seed(r_seed)
    random_labels = np.random.randint(0, 3 + 1, size=len(true_labels))
    diffs = abs(true_labels - random_labels)
    penalties_full_chance.append(diffs.mean())

print(
    'Penalty alpha .05 cut off without distribution'
    f' knowledge: {np.mean(penalties_full_chance)}')

Penalty alpha .05 cut off without distribution knowledge: 1.1336091549295775


In [323]:
perm_dir = os.path.join(
    find_onedrive_path('results'),
    'predictions', 'permutations')
perm_fname = f'RF_full_dev_data_{n_permutations}perms_means.csv'
mean_penalties = np.loadtxt(os.path.join(perm_dir, perm_fname))
sign_05 = round(np.percentile(mean_penalties, 5), 2)

plt.rc('font', size=14)
plt.hist(mean_penalties)
plt.title(f'Alpha .05 significance cut off: {sign_05}', size=14)
ymin, ymax = plt.ylim()
plt.vlines(sign_05, ymin=ymin, ymax=ymax, color='r', ls='--')
plt.close()

In [276]:
importlib.reload(utils_dataManagement)
from retap_utils.utils_dataManagement import find_onedrive_path

save_dir = os.path.join(
    find_onedrive_path('results'),
    'predictions', 'permutations')
fname = f'RF_full_dev_data_{n_permutations}perms_lists'

if not os.path.exists(save_dir): os.makedirs(save_dir)

np.savetxt(
    os.path.join(save_dir, f'{fname}.csv'),
    random_penalties['lists'], delimiter=',',
)


## 3) Clustering & Classifying

- Candidate vetors based on descriptives and concept
    - nTaps
    - freq
    - upVelo sum [std-dev + coefVar]
    - impact RMS [coefVar + stddev]
    - tapRMS and impactRMS [sum]
    - 
- include per run (array tap-features): sum, mean, stddev, trend_slope

- Cluster on UPDRS 4?

### 3a) Clustering

In [10]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# try: K-shape (sklearn), Laio 2014

In [71]:
importlib.reload(plot_cluster)

n_clusters=4
center_incl = 'all'
sel_feats = True


### VISUALISE AGAINST SUBS !! AND CONDITIONS


traces, feats = pred_prep.select_traces_and_feats(
    ftClass,
    center=center_incl,
    use_sel_fts=sel_feats,
)
X, y = pred_prep.create_X_y_vectors(
    ftClass,
    incl_traces=traces,
    incl_feats=feats,
    to_norm=False,
)

figname = (
    f'retap_{n_clusters}clusters_'
    f'{center_incl}'
)
if sel_feats: figname += '_selFeats'
else: figname += '_allFeats'

plot_cluster.plot_cluster_kMeans(
    X=X, y=y,
    n_clusters=n_clusters,
    use_pca=True,
    random_state=27,
    figsave_name=figname,
    figsave_dir=os.path.join(
        utils_dataManagement.find_onedrive_path('figures'),
        'clustering',
    ),
    show=False,
)

c:\Users\habetsj\Anaconda3\envs\retap\lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


#### MANOVA

- normality assumption violated (Shapiro test highly significant)
- for every a priori selected feature: present difference between sub-score-groups is a Kruskal-Wallis test (non-parametric One-Way ANOVA alternative)
- differences between two sub groups within a feature is a non-parametric test of two groups of quantitative values (likely varying lengths): Mann-Whitney-U
- in total: correct alpha for number of repeated measures on specific level

In [ ]:
from scipy.stats import shapiro
for col in np.arange(X.shape[1]):
    print(feats[col], shapiro(X[:, col]))

In [90]:
from statsmodels.multivariate.manova import MANOVA

stat_data = np.concatenate([X, y.reshape((len(y), 1))], axis=1)
manova_df = pd.DataFrame(
    data=stat_data,
    columns=feats + ['subscore'],
)
maov = MANOVA.from_formula(
    'nTaps + freq + mean_intraTapInt + coefVar_intraTapInt + IQR_jerkiness +'
    ' mean_raise_velocity + mean_tapRMSnrm ~ subscore ',
    # 'mean_jerkiness_smooth + IQR_jerkiness_smooth ~ subscore',
    data=manova_df,
)
print(maov.mv_test())

                  Multivariate linear model
                                                              
--------------------------------------------------------------
       Intercept        Value  Num DF  Den DF  F Value  Pr > F
--------------------------------------------------------------
          Wilks' lambda 0.1239 7.0000 365.0000 368.6918 0.0000
         Pillai's trace 0.8761 7.0000 365.0000 368.6918 0.0000
 Hotelling-Lawley trace 7.0708 7.0000 365.0000 368.6918 0.0000
    Roy's greatest root 7.0708 7.0000 365.0000 368.6918 0.0000
--------------------------------------------------------------
                                                              
--------------------------------------------------------------
         subscore        Value  Num DF  Den DF  F Value Pr > F
--------------------------------------------------------------
           Wilks' lambda 0.7803 7.0000 365.0000 14.6821 0.0000
          Pillai's trace 0.2197 7.0000 365.0000 14.6821 0.0000
  Hotelling

In [126]:
from scipy.stats import kruskal
importlib.reload(pred_prep)

mask_scores = True

traces, feats = pred_prep.select_traces_and_feats(
    ftClass,
    center=center_incl,
    use_sel_fts=sel_feats,
)
X, y = pred_prep.create_X_y_vectors(
    ftClass,
    incl_traces=traces,
    incl_feats=feats,
    to_norm=False,
)
n_groups = 5
if mask_scores:
    # UPDRS 4 -> 3 merge
    mask = y == 4
    y[mask] = 3
    # UPDRS 0 -> 1 merge
    mask = y == 0
    y[mask] = 1

    n_groups = 3

stat_data = np.concatenate([X, y.reshape((len(y), 1))], axis=1)
stat_df = pd.DataFrame(
    data=stat_data,
    columns=feats + ['subscore'],
)

stat_fts = [
    'freq', 'coefVar_intraTapInt', 'mean_jerkiness', 'coefVar_jerkiness',
    'mean_tapRMSnrm', 'coefVar_tapRMSnrm', 'slope_tapRMSnrm'
]
alpha = .05 / len(stat_fts)
for ft in stat_fts:
    tempft = stat_df[~np.isnan(stat_df[ft])]

    
    if mask_scores:
        groups = [
            tempft[ft][tempft['subscore'] == s].reset_index(drop=True)
            for s in np.arange(1, n_groups + 1)
        ]
        krusk_stat, krusk_p = kruskal(
            groups[0], groups[1], groups[2], 
        )
    else:
        groups = [
            tempft[ft][tempft['subscore'] == s].reset_index(drop=True)
            for s in np.arange(n_groups)
        ]
        krusk_stat, krusk_p = kruskal(
            groups[0], groups[1], groups[2], 
            groups[3], groups[4]
        )
    print(f'\n{ft}: \n\tGroup level sign. difference (Kruskal'
        f' Test): {krusk_p < alpha} (p = {np.round(krusk_p, 6)})\n')
    for g in np.arange(n_groups - 1):

        mnwu_rho, mnwu_p = mannwhitneyu(groups[g], groups[g + 1])
        print(f'\tupdrs {g} vs {g + 1} sign, (Mann-Whitney-U): '
            f'{mnwu_p < (alpha / (n_groups - 1))} (p = {np.round(mnwu_p, 6)})')



freq: 
	Group level sign. difference (Kruskal Test): True (p = 3.8e-05)

	updrs 0 vs 1 sign, (Spearman): False (p = 0.485159)
	updrs 1 vs 2 sign, (Spearman): True (p = 0.00025)

coefVar_intraTapInt: 
	Group level sign. difference (Kruskal Test): True (p = 0.0)

	updrs 0 vs 1 sign, (Spearman): False (p = 0.084536)
	updrs 1 vs 2 sign, (Spearman): True (p = 4.8e-05)

mean_jerkiness: 
	Group level sign. difference (Kruskal Test): False (p = 0.331604)

	updrs 0 vs 1 sign, (Spearman): False (p = 0.375883)
	updrs 1 vs 2 sign, (Spearman): False (p = 0.140139)

coefVar_jerkiness: 
	Group level sign. difference (Kruskal Test): False (p = 0.719329)

	updrs 0 vs 1 sign, (Spearman): False (p = 0.507749)
	updrs 1 vs 2 sign, (Spearman): False (p = 0.490786)

mean_tapRMSnrm: 
	Group level sign. difference (Kruskal Test): True (p = 3e-06)

	updrs 0 vs 1 sign, (Spearman): False (p = 0.017736)
	updrs 1 vs 2 sign, (Spearman): True (p = 0.00162)

coefVar_tapRMSnrm: 
	Group level sign. difference (Kruskal 

In [119]:
from scipy.stats import mannwhitneyu

In [ ]:
X, y = pred_prep.create_X_y_vectors(
    ftClass,
    incl_traces=traces,
    incl_feats=feats,
    to_norm=False,
)

# UPDRS 4 -> 3 merge
mask = y == 4
y[mask] = 3
# UPDRS 0 -> 1 merge
mask = y == 0
y[mask] = 1
### 3c. Classification

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

from sklearn.metrics import classification_report
from sklearn.metrics import multilabel_confusion_matrix


#### Classifying UPDRS 0 - 1 - 2 - 3 - 4

In [37]:
traces, feats = pred_prep.select_traces_and_feats(
    ftClass,
    center='all',
    use_sel_fts=True,
)
X, y = pred_prep.create_X_y_vectors(
    ftClass,
    incl_traces=traces,
    incl_feats=feats,
    to_norm=False,
)
print(X.shape)

# use random outcome labels (equal distribution over scores)
random_y = np.random.randint(0, 5, size=X.shape[0])
y = random_y
# use shuffled outcome labels (same distribution)
np.random.seed(27)
# np.random.shuffle(y)

# UPDRS 4 -> 3 merge
# mask = y == 4
# y[mask] = 3
# # UPDRS 0 -> 1 merge
# mask = y == 0
# y[mask] = 1

lda = LDA()
lda.fit(X, y)

#Define method to evaluate model
cv = RepeatedStratifiedKFold(n_splits=8, n_repeats=10, random_state=1)

#evaluate model
scores = cross_val_score(lda, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print(np.mean(scores)) 
print(scores)

(373, 12)
0.19776827012025905
[0.17021277 0.14893617 0.21276596 0.21276596 0.19148936 0.15217391
 0.13043478 0.2173913  0.14893617 0.08510638 0.17021277 0.23404255
 0.17021277 0.30434783 0.23913043 0.2826087  0.14893617 0.25531915
 0.12765957 0.17021277 0.19148936 0.17391304 0.2173913  0.10869565
 0.27659574 0.12765957 0.21276596 0.17021277 0.29787234 0.26086957
 0.06521739 0.19565217 0.25531915 0.19148936 0.29787234 0.23404255
 0.19148936 0.17391304 0.17391304 0.13043478 0.17021277 0.23404255
 0.23404255 0.29787234 0.19148936 0.13043478 0.19565217 0.26086957
 0.14893617 0.21276596 0.31914894 0.25531915 0.10638298 0.15217391
 0.17391304 0.23913043 0.21276596 0.21276596 0.14893617 0.25531915
 0.27659574 0.15217391 0.17391304 0.13043478 0.21276596 0.19148936
 0.29787234 0.19148936 0.19148936 0.08695652 0.23913043 0.26086957
 0.19148936 0.23404255 0.27659574 0.14893617 0.10638298 0.19565217
 0.2173913  0.17391304]


In [18]:
# CLassification Settings

X, y = pred_prep.create_X_y_vectors(
    ftClass,
    incl_traces=traces,
    incl_feats=feats,
    to_norm=False,
)
# print(f'INCLUDED FEATURE SPACE: {X.shape}')

nFolds = 10

# UPDRS 4 -> 3 merge
mask = y == 4
y[mask] = 3
# UPDRS 0 -> 1 merge
mask = y == 0
y[mask] = 1


# Shuffle order
# allData = np.hstack((X, y))
# X_shf = allData[:, :14]
# y_shf = allData[:, 14]

# np.random.shuffle(allData)

skf = StratifiedKFold(n_splits=nFolds,)
skf.get_n_splits(X, y)
# clf = LinearSVC(
#         penalty='l2',
#     C=1.0,
#     multi_class='ovr',
#     max_iter=10000,
# )
clf = LogisticRegression(
    random_state=0, 
    solver='liblinear',
    multi_class='ovr',
)

shuffled_accs = []
np.random.seed(27)
# for r_state in np.random.randint(100, size=1000):

    # np.random.seed(r_state)
    # # y_shuffled = y.copy()
    # # np.random.shuffle(y_shuffled)
    # np.random.shuffle(y)

y_pred, y_true = {}, {}
all_accs = []
for F, (train_index, test_index) in enumerate(
    skf.split(X, y)
):
    # print(f'\nLinear Support Vector, fold #{F}')
    # print(f'\nLogistic Regression, fold #{F}')
    # print(
    #     f'\tn train: {train_index.shape}, '
    #     f'n test: {test_index.shape}')

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # print('\tcount for y-labels in test set\n',
    #     np.array(np.unique(y_test, return_counts=True)).T)
    
    clf = clf
    clf.fit(X=X_train, y=y_train)
    acc = clf.score(X=X_test, y=y_test)
    # print(
    #     f'accuracy for Fold {F}: {acc}')
    all_accs.append(acc)
    
    # save predictions for posthoc analysis and conf matrix
    y_pred[F] = clf.predict(X=X_test)
    y_true[F] = y_test
    # print(multilabel_confusion_matrix(y_true[F], y_pred[F]))

        
print(f'Overall mean Accuracy: {np.mean(all_accs)}\n')
    # shuffled_accs.append(np.mean(all_accs))

# print('grand mean', np.mean(shuffled_accs))


Overall mean Accuracy: 0.534068278805121



In [132]:
print(f'Overall mean Accuracy: {np.mean(all_accs)}')


Overall mean Accuracy: 0.4047700754975978


#### Boolean Classifying (UPDRS 0 or 4 vs The Rest)

In [83]:
# CLassification Settings
nFolds = 4
score_to_predict = 0

# Shuffle order
X = Xdf.values
y_bool = y == score_to_predict

allData = np.hstack((X, y_bool))

np.random.seed(27)
np.random.shuffle(allData)

X_shf = allData[:, :14]
y_shf = allData[:, 14]

skf = StratifiedKFold(n_splits=nFolds,)
skf.get_n_splits(X_shf, y_shf)

y_pred, y_true = {}, {}
print(
    'Classification of UPDRS subscore '
    f'{score_to_predict} versus thre rest')
for F, (train_index, test_index) in enumerate(
    skf.split(X, y)
):
    print(f'\nLinear Support Vector, fold #{F}')

    X_train, X_test = X_shf[train_index], X_shf[test_index]
    y_train, y_test = y_shf[train_index], y_shf[test_index]

    clf = LinearSVC(penalty='l2', C=1.0,)
    clf.fit(X=X_train, y=y_train)
    print(f'Accuracy: {clf.score(X=X_test, y=y_test)}')
    # for own scoring
    y_pred[F] = clf.predict(X=X_test)
    y_true[F] = y_test
    print(classification_report(y_true[F], y_pred[F]))
    



Classification of UPDRS subscore 0 versus thre rest

Linear Support Vector, fold #0


ValueError: Unknown label type: 'continuous'